In [182]:
import pandas as pd

# Importando relatorio de frequencia

In [183]:
relatorio_frequencia = pd.read_csv('relatorio_de_frequencia.csv', sep=';', encoding='latin-1')


In [184]:
relatorio_frequencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10163 entries, 0 to 10162
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Paciente          10163 non-null  object
 1   Criacao do plano  10163 non-null  object
 2   Procedimento      10163 non-null  object
 3   Status            10163 non-null  object
 4   Sessoes           10163 non-null  int64 
 5   Realizadas        10163 non-null  int64 
 6   Restantes         10163 non-null  int64 
 7   Orcamento         10163 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 635.3+ KB


In [ ]:
relatorio_frequencia.head()

In [ ]:
relatorio_frequencia.describe()

Parece que você carregou com sucesso um DataFrame no pandas. O DataFrame possui 8638 entradas e 8 colunas. Aqui está uma breve descrição das colunas:

-   **Paciente**: Nome do paciente.
-   **Criacao do plano**: Data em que o plano foi criado.
-   **Procedimento**: Descrição do procedimento.
-   **Status**: Status do procedimento (por exemplo, "Aguardando" ou "Realizado").
-   **Sessoes**: Número total de sessões planejadas para o procedimento.
-   **Realizadas**: Número de sessões já realizadas do procedimento.
-   **Restantes**: Número de sessões restantes para o procedimento.
-   **Orcamento**: Valor do orçamento para o procedimento.

# Importando relatório de precos_tabelados

In [187]:
precos_tabelados=pd.read_csv('precos_tabelados_2.csv', sep=';', encoding='latin-1')

In [188]:
precos_tabelados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Procedimento      238 non-null    object 
 1   Via de aplicação  238 non-null    object 
 2   Composição        238 non-null    float64
 3   Honorários        238 non-null    float64
 4   Valor             238 non-null    float64
dtypes: float64(3), object(2)
memory usage: 9.4+ KB


In [ ]:
precos_tabelados.head()

In [190]:
precos_tabelados.describe()

,Composição,Honorários,Valor
count,238.000000,238.000000,238.000000
mean,105.735756,1256.744706,1362.480462
std,327.551806,2789.106312,2815.462371
min,0.000000,0.000000,0.000000
25%,0.000000,209.617500,279.985000
50%,24.200000,336.060000,442.990000
75%,103.892500,456.660000,577.842500
max,4050.000000,20000.000000,20000.000000


# Fazendo merge para procedimentos normalizados em maiúsculo.

In [191]:
relatorio_frequencia['Procedimento']=relatorio_frequencia['Procedimento'].str.upper()
precos_tabelados['Procedimento']=precos_tabelados['Procedimento'].str.upper()

frequencia_precos=pd.merge(relatorio_frequencia, precos_tabelados, how='left', on='Procedimento')

# Observando procedimentos ausentes

In [ ]:
# Lista de procedimentos presentes na tabela 'frequencia_precos'
procedimentos_frequencia = frequencia_precos['Procedimento'].unique()

# Lista de procedimentos presentes na tabela 'precos_tabelados'
procedimentos_tabelados = precos_tabelados['Procedimento'].unique()

# Filtrar os procedimentos que estão em 'frequencia_precos', mas não em 'precos_tabelados'
procedimentos_ausentes = [procedimento for procedimento in procedimentos_frequencia if procedimento not in procedimentos_tabelados]

# Exibir os procedimentos ausentes
print(procedimentos_ausentes)


# Fazendo associação para valores nulos e em seguida fazendo join com fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

# List de procedimentos presentes na tabela 'frequencia_precos'
procedimentos_frequencia = frequencia_precos['Procedimento'].unique()

# Iterando pelas linhas onde 'Valor' é nulo após o join e completar com os valores correspondentes usando fuzzywuzzy
for index, row in frequencia_precos.iterrows():
    if pd.isnull(row['Valor']):
        procedimento_frequencia = row['Procedimento']
        for procedimento_tabelado in procedimentos_tabelados:
            similaridade = fuzz.token_sort_ratio(procedimento_frequencia, procedimento_tabelado)
            if similaridade >= 80:  # Definir o limite de similaridade desejado
                valor_tabelado = precos_tabelados.loc[precos_tabelados['Procedimento'] == procedimento_tabelado, 'Valor'].iloc[0]
                # Preencher o valor nulo com o valor correspondente da tabela 'precos_tabelados'
                frequencia_precos.at[index, 'Valor'] = valor_tabelado
                break

# Exibindo o DataFrame 'frequencia_precos' com os valores completados
print(frequencia_precos)
frequencia_precos_atualizado=frequencia_precos

# Separando novos pacientess

In [ ]:
from datetime import datetime

# Converter a coluna 'Criacao do plano' para o formato datetime
frequencia_precos['Criacao do plano'] = pd.to_datetime(frequencia_precos['Criacao do plano'], format='%d/%m/%Y')

# Ordenar o DataFrame por paciente e data em ordem crescente
frequencia_precos.sort_values(by=['Paciente', 'Criacao do plano'], inplace=True)
frequencia_precos_com_data=frequencia_precos
# Criar uma nova coluna chamada 'Primeira Data de Procedimento' e preenchê-la com a primeira data de cada paciente
frequencia_precos_com_data['Primeira Data de Procedimento'] = frequencia_precos_com_data.groupby('Paciente')['Criacao do plano'].transform('min')

# Filtrar apenas as linhas com a primeira data no mês de julho de 2023
frequencia_precos_com_data = frequencia_precos_com_data[(frequencia_precos_com_data['Primeira Data de Procedimento'].dt.year == 2023) & (frequencia_precos_com_data['Primeira Data de Procedimento'].dt.month == 7)]

# Exibir a tabela resultante
print(frequencia_precos_com_data)

# Gerando Pivot table dos novos pacientes

In [ ]:
# Criar a tabela dinâmica (pivot table) com todas as colunas como índices
pivot_table_all_index_month = frequencia_precos_com_data.pivot_table(index=['Paciente', 'Criacao do plano', 'Procedimento', 'Status'], 
                                                     aggfunc='sum',
                                                     fill_value=0)

# Exibir a tabela dinâmica
print(pivot_table_all_index_month)


In [196]:
pivot_table_all_index_month.to_excel('pivot_pacientes_first_month.xlsx')

# Pivot table geral para pacientes

In [ ]:
# Criar a tabela dinâmica (pivot table) com todas as colunas como índices
pivot_table_all_index = frequencia_precos.pivot_table(index=['Paciente', 'Criacao do plano', 'Procedimento', 'Status'], 
                                                     aggfunc='sum',
                                                     fill_value=0)

# Exibir a tabela dinâmica
print(pivot_table_all_index)


In [ ]:
# Convertendo a coluna 'Criacao do plano' para datetime
frequencia_precos['Criacao do plano'] = pd.to_datetime(frequencia_precos['Criacao do plano'])

# Criando as colunas 'Ano' e 'Mes'
frequencia_precos['Ano'] = frequencia_precos['Criacao do plano'].dt.year
frequencia_precos['Mes'] = frequencia_precos['Criacao do plano'].dt.month

# Filtrando o DataFrame
frequencia_precos_filtrado = frequencia_precos[(frequencia_precos['Ano'] == 2023) & (frequencia_precos['Mes'] == 7) & (frequencia_precos['Status'] == 'Finalizado')]

# Criando a tabela dinâmica
pivot_table = frequencia_precos_filtrado.pivot_table(index=['Paciente', 'Criacao do plano', 'Procedimento', 'Status'], 
                                                     aggfunc='sum',
                                                     fill_value=0)

# Exibindo a tabela dinâmica
print(pivot_table)


In [199]:
pivot_table_all_index.to_excel('pivot_pacientes.xlsx')

In [200]:
# Convertendo a coluna 'Criacao do plano' para datetime
frequencia_precos['Criacao do plano'] = pd.to_datetime(frequencia_precos['Criacao do plano'])

# Criando as colunas 'Ano' e 'Mes'
frequencia_precos['Ano'] = frequencia_precos['Criacao do plano'].dt.year
frequencia_precos['Mes'] = frequencia_precos['Criacao do plano'].dt.month

# Identificando pacientes que realizaram pelo menos um procedimento em Julho de 2023
pacientes_julho_2023 = frequencia_precos[(frequencia_precos['Ano'] == 2023) & (frequencia_precos['Mes'] == 7)]['Paciente'].unique()


In [ ]:
# Filtrando o DataFrame para incluir apenas procedimentos realizados por pacientes identificados
frequencia_precos_filtrado = frequencia_precos[frequencia_precos['Paciente'].isin(pacientes_julho_2023) & (frequencia_precos['Status'] == 'Finalizado')]

# Criando a tabela dinâmica
pivot_table = frequencia_precos_filtrado.pivot_table(index=['Paciente', 'Criacao do plano', 'Procedimento', 'Status'], 
                                                     aggfunc='sum',
                                                     fill_value=0)

# Exibindo a tabela dinâmica
print(pivot_table)
pivot_table.to_excel('pacotes_finalizados_07_2023.xlsx')






# Pivot table para procedimentos

In [ ]:
# Criar a tabela dinâmica (pivot table) com todas as colunas como índices
pivot_table_all_index_month_procedimentos = frequencia_precos_atualizado.pivot_table(index=['Procedimento', 'Criacao do plano', 'Paciente', 'Status'], 
                                                     aggfunc='sum',
                                                     fill_value=0)

# Exibir a tabela dinâmica
print(pivot_table_all_index_month_procedimentos)

pivot_table_all_index_month_procedimentos.to_excel('pivot_procedimentos.xlsx')


# Relatórios mensais (Todos pacientes)

In [205]:
frequencia_precos_atualizado['Criacao do plano'] = pd.to_datetime(frequencia_precos_atualizado['Criacao do plano'], format='%d/%m/%Y')

frequencia_precos_atualizado = frequencia_precos_atualizado[
    (frequencia_precos_atualizado['Criacao do plano'] >= '2023-06-25') &
    (frequencia_precos_atualizado['Criacao do plano'] <= '2023-07-24')
]

In [206]:
# Criar a tabela dinâmica (pivot table) com todas as colunas como índices
pivot_table_all_index_month_todos_pacientes = frequencia_precos_atualizado.pivot_table(index=['Paciente', 'Criacao do plano', 'Procedimento', 'Status'], 
                                                     aggfunc='sum',
                                                     fill_value=0)

# Exibir a tabela dinâmica
pivot_table_all_index_month_todos_pacientes.to_excel('relatorio_de_pacientes_no_mes.xlsx')


C:\Users\gabri\AppData\Local\Temp\ipykernel_31532\112398506.py:2: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  pivot_table_all_index_month_todos_pacientes = frequencia_precos_atualizado.pivot_table(index=['Paciente', 'Criacao do plano', 'Procedimento', 'Status'],


In [207]:
pivot_procedimentos_relatorio=frequencia_precos_atualizado.pivot_table(index='Procedimento', values=['Sessoes', 'Valor'], aggfunc='sum')

In [208]:
pivot_procedimentos_relatorio.to_excel('tabela_sessoes_lucro_procedimentos_mensais.xlsx')

# Procedimentos realizados no mes por paciente

In [209]:
pivot_table_all_index_month = frequencia_precos_atualizado.pivot_table(index=['Paciente', 'Criacao do plano', 'Procedimento', 'Status'], 
                                                     aggfunc='sum',
                                                     fill_value=0)
pivot_table_all_index_month.to_excel('pivot_pacientes_month.xlsx') 

C:\Users\gabri\AppData\Local\Temp\ipykernel_31532\2616095150.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  pivot_table_all_index_month = frequencia_precos_atualizado.pivot_table(index=['Paciente', 'Criacao do plano', 'Procedimento', 'Status'],


# Procedimentos desejados

In [ ]:
import re

procedimentos_desejados = [
    #
]

# Criar uma tabela dinâmica para cada procedimento desejado
for procedimento in procedimentos_desejados:
    # Filtrar o DataFrame 'frequencia_precos_atualizado' para o procedimento atual
    df = frequencia_precos_atualizado[frequencia_precos_atualizado['Procedimento'].str.contains(procedimento, case=False, na=False)]
    
    # Imprimir a tabela dinâmica para o procedimento atual
    pivot_table_procedimento = df.pivot_table(index=['Paciente', 'Criacao do plano', 'Procedimento', 'Status'],
                                                          aggfunc='sum',
                                                          fill_value=0)
    nome_arquivo = re.sub(r'[|/\\]', '-', procedimento)
    pivot_table_procedimento.to_excel(f'{nome_arquivo}.xlsx', index=True)
    